In [ ]:
import traceback
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats as stats
from pprint import pprint
from scipy.special import logsumexp
from sklearn.cluster import KMeans
from scipy import interpolate

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
rng = np.random.RandomState(42)

source : http://approximateinference.org/2017/accepted/Zinkov2017.pdf
source : https://towardsdatascience.com/implement-expectation-maximization-em-algorithm-in-python-from-scratch-f1278d1b9137

In [ ]:
df = pd.read_csv("../data/X_station_day.csv")

In [ ]:
df.rename(columns={"precipitations": "current_precipitations"}, inplace=True)

## EM
 M-step :

$$ \pi = \frac{\sum_{i=1}^{n}\mathcal{Q}(y^i=1|x^i)}{n} $$

$$ \mu_j = \frac{\sum_{i=1}^{n}x^i\mathcal{Q}(y^i=j|x^i)}{\sum_{i=1}^{n}\mathcal{Q}(y^i=j|x^i)} $$

$$ \Sigma_j = \frac{\sum_{i=1}^{n}\mathcal{Q}(y^i=j|x^i)(x^i-\mu_j)(x^i-\mu_j)^T}{\sum_{i=1}^{n}\mathcal{Q}(y^i=j|x^i)} $$

## 1 dimension

In [ ]:
class EM_1D():
    def __init__(self, x: np.ndarray, nb_dist=2, split_ratio=0.1, threshold=0.01, minimum_sigma=1e-5, talk=False):
        self.talk = talk  # whether to display information or not
        self.threshold = threshold  # threshold ratio to trigger stop_fitting
        self.minimum_sigma = minimum_sigma  # to avoid sigma being 0
        self.stop_fitting = False  # stop fitting if maximum iterations reached or threshold passed
        self.nb_dist = nb_dist  # number of distributions to fit

        self.split_ratio = split_ratio  # part of x to keep to initialize the parameters
        self.x_all = x  # backup of whole x data
        self.idx_sample = None  # index of x_sample for initialization
        self.x_sample = None  # sample for initialization
        self.x = None  # training part of x
        self.split_x()  # fill x_sample, idx_sample and x

        self.mu = None  # average
        self.sigma = None  # standard deviation
        self.w = None  # weights
        self.init_params()  # initialize the above

        self.history = None  # history
        self.init_history()  # initialize history

        self.r = None  # declare r to anticipate fit()

        self.initial_record = None  # declare variable, used in average_fit()
        self.history_record = None  # declare variable, used in average_fit()
        self.clustered_distributions = None  # declare variable, used in average_fit()

        self.colors = ["blue", "red", "orange", "green", "magenta"]

    def split_x(self):
        self.idx_sample = rng.choice(np.arange(self.x_all.shape[0]),
                                     replace=False,
                                     size=int(self.x_all.shape[0] * self.split_ratio))
        self.x_sample = self.x_all[self.idx_sample]
        self.x = np.delete(self.x_all, self.idx_sample, axis=0)

    def init_params(self):
        mu = rng.normal(loc=np.mean(self.x_sample),
                        scale=5,
                        size=self.nb_dist)
        sigma = np.abs(rng.normal(loc=np.sqrt(np.var(self.x_sample)),
                                  scale=2,
                                  size=self.nb_dist))
        w = np.abs(rng.normal(loc=1,
                              scale=0.2,
                              size=self.nb_dist))
        w /= w.sum()
        if self.talk:
            print("Initial parameters :")
            print("\t - mu :    {}".format(np.round(mu, 2)))
            print("\t - sigma : {}".format(np.round(sigma, 2)))
            print("\t - w :     {}".format(np.round(w, 2)))
        self.mu, self.sigma, self.w = mu, sigma, w

    def init_history(self):
        history = {"mu": [self.mu],
                   "sigma": [self.sigma],
                   "w": [self.w]}
        self.history = history

    def update_h(self):
        self.history["mu"].append(self.mu)
        self.history["sigma"].append(self.sigma)
        self.history["w"].append(self.w)

    def e_step(self):
        x = self.x[None, :]
        mu = self.mu[:, None]
        sigma = self.sigma[:, None]
        # r is the likelyhood of each point of x given the normal distribution N(mu,sigma)
        self.r = stats.norm.pdf(x, mu, sigma)
        self.r = self.r / self.r.sum(axis=0)

    def m_step(self):
        n = self.x.shape[0]
        self.w = np.sum(self.r, axis=1) / n
        self.mu = np.sum(self.x * self.r, axis=1) / np.sum(self.r, axis=1)
        sigma = []
        for j in range(self.nb_dist):
            sigma.append(
                np.sqrt(np.sum(self.r[j, :] * (self.x - self.mu[j]) * (self.x - self.mu[j])) / np.sum(self.r[j, :])))
            if sigma[-1] == 0:
                print(
                    "WARNING : sigma == 0, set sigma to minimum_sigma = {}".format(self.minimum_sigma))
                sigma[-1] = self.minimum_sigma
        self.sigma = np.array(sigma)

    def fit(self, max_iters=100, plot=False):
        self.stop_fitting = False
        for i in range(max_iters):
            if self.talk and i % int(max_iters // 100 + 1) == 0:
                print("{} %".format(int(i / max_iters * 100)).rjust(5), end="\r")

            self.e_step()
            self.m_step()
            self.update_h()

            self.check_change()
            if self.stop_fitting:
                break

        if self.talk:
            print("\n")
            if self.stop_fitting:
                print("Threshold reached, less than {} % difference.".format(self.threshold * 100))
            else:
                print("100 %")
            print("Final parameters :\n\tmu :    {}\n\tsigma : {}\n\tw :     {}".format(self.history["mu"][-1],
                                                                                        self.history["sigma"][-1],
                                                                                        self.history["w"][-1]))

        if plot:
            self.plot_progression()

    def check_change(self):
        mu_old = self.history["mu"][-2]
        sigma_old = self.history["sigma"][-2]
        w_old = self.history["w"][-2]
        if any(self.mu == 0):
            delta_mu = np.max(np.abs((mu_old - self.mu) / (self.mu + 1)))
        else:
            delta_mu = np.max(np.abs((mu_old - self.mu) / self.mu))
        delta_sigma = np.max(np.abs((sigma_old - self.sigma) / self.sigma))
        if any(self.w == 0):
            delta_w = np.max(np.abs((w_old - self.w) / (self.w + 1)))
        else:
            delta_w = np.max(np.abs((w_old - self.w) / self.w))
        delta = max(delta_mu, delta_sigma, delta_w)
        if delta < self.threshold:
            self.stop_fitting = True

    def average_fit(self, runs=3, max_iters=100, plot=True, plot_title=""):
        """
        to avoid initialization issues, average over [runs] fit() and different initializations.
        :param plot_title: title of the plot of plot
        :param runs: number of fit() to execute
        :param max_iters: maximum number of iterations for each fit()
        :param plot: plot the final result, average of all fit()

        updates to history to every parameters fo every fit() (for plotting purpose) and parameters to average of every run.
        """

        initial_record = pd.DataFrame(columns=["mu", "sigma", "w"])
        history_record = pd.DataFrame(columns=["mu", "sigma", "w"])
        for i in range(runs):
            if self.talk:
                print("\nRun number {} over {}.".format(i, runs))
            # reset everything at each run
            self.init_params()
            initial_record = initial_record.append(pd.DataFrame({"mu": self.mu,
                                                                 "sigma": self.sigma,
                                                                 "w": self.w}),
                                                   ignore_index=True)
            self.init_history()
            self.split_x()

            self.fit(max_iters=max_iters)

            for j in range(self.nb_dist):
                history_record = history_record.append(pd.DataFrame({"mu": [self.history["mu"][-1][j]],
                                                                     "sigma": [self.history["sigma"][-1][j]],
                                                                     "w": [self.history["w"][-1][j]]}),
                                                       ignore_index=True)

        history_record_scaled = (history_record - history_record.min()) / (history_record.max() - history_record.min())
        kmeans = KMeans(init="random", n_clusters=self.nb_dist, n_init=10, max_iter=300, random_state=42)
        history_record["cluster"] = kmeans.fit_predict(history_record_scaled)
        initial_record["cluster"] = history_record["cluster"]
        self.initial_record = initial_record.sort_values(by="cluster", ignore_index=True)
        self.history_record = history_record.sort_values(by="cluster", ignore_index=True)

        clustered_distributions = pd.DataFrame(columns=["mu", "sigma", "w"])
        for i in range(self.nb_dist):
            mu = history_record[history_record['cluster'] == i]["mu"].mean()
            sigma = history_record[history_record['cluster'] == i]["sigma"].mean()
            w = history_record[history_record['cluster'] == i]["w"].mean()
            clustered_distributions = clustered_distributions.append(
                pd.DataFrame({"mu": [mu],
                              "sigma": [sigma],
                              "w": [w]}),
                ignore_index=True
            )
        clustered_distributions["w"] /= clustered_distributions["w"].sum()

        self.clustered_distributions = clustered_distributions

        if self.talk:
            print("\n\n", "=" * 25, "\nFinal average parameters :\n\tmu :    {}\n\tsigma : {}\n\tw :     {}".format(
                self.clustered_distributions["mu"].tolist(),
                self.clustered_distributions["sigma"].tolist(),
                self.clustered_distributions["w"].tolist()))

        if plot:
            if not plot_title:
                plot_title = "Average fit over {} runs".format(runs)
            self.plot_average(title=plot_title)

    ###########################
    ### POT FUNCTIONS BELOW ###
    ###########################

    def plot_average(self, figsize=(20, 6), title="specific_plot", save=True):
        abscisse = np.linspace(self.x_all.min(), self.x_all.max(), 1001)

        plt.figure(figsize=figsize)
        plt.suptitle(title)

        # hist plots
        ax = plt.subplot2grid(shape=(1, 3), loc=(0, 0))
        ax.set_title("Initial distributions")
        sns.histplot(self.x_all, bins=50, alpha=0.2, color="cyan", stat="probability", ax=ax)

        ax0 = plt.subplot2grid(shape=(1, 3), loc=(0, 1))
        ax0.set_title("All distributions after convergence or max_iters")
        sns.histplot(self.x_all, bins=50, alpha=0.2, color="cyan", stat="probability", ax=ax0)

        ax1 = plt.subplot2grid(shape=(1, 3), loc=(0, 2))
        ax1.set_title("Averaged distributions by {}-means".format(self.nb_dist))
        mu_s = self.clustered_distributions["mu"]
        sigma_s = self.clustered_distributions["sigma"]
        w_s = self.clustered_distributions["w"]
        sns.histplot(self.x_all, bins=50, alpha=0.2, color="cyan", stat="probability", ax=ax1)

        # find y_limit
        y_lim_top = min(1, ax.set_ylim(auto=True)[1], ax0.set_ylim(auto=True)[1], ax1.set_ylim(auto=True)[1])

        # line plots
        for j in range(len(self.initial_record)):
            ax.plot(abscisse,
                    self.initial_record.loc[j, "w"] * stats.norm.pdf(abscisse,
                                                                     self.initial_record.loc[j, "mu"],
                                                                     self.initial_record.loc[j, "sigma"]),
                    color=self.colors[self.initial_record.loc[j, "cluster"]])

        for j in range(len(self.history_record)):
            ax0.plot(abscisse,
                     self.history_record.loc[j, "w"] * stats.norm.pdf(abscisse,
                                                                      self.history_record.loc[j, "mu"],
                                                                      self.history_record.loc[j, "sigma"]),
                     color=self.colors[self.history_record.loc[j, "cluster"]])

        for j in range(self.nb_dist):
            ax1.plot(abscisse,
                     w_s[j] * stats.norm.pdf(abscisse,
                                             mu_s[j],
                                             sigma_s[j]),
                     color=self.colors[j])

        # set y_limit
        ax.set_ylim(top=y_lim_top)
        ax0.set_ylim(top=y_lim_top)
        ax1.set_ylim(top=y_lim_top)

        plt.legend([f"cluster_{i}" for i in range(self.nb_dist)])

        if save:
            plt.savefig("../../images/1D/" + title.replace(" ", "_") + ".jpg")
        plt.show()

    def plot_step(self, step_index, figsize=(10, 7)):
        mu = self.history["mu"][step_index]
        sigma = self.history["sigma"][step_index]
        w = self.history["w"][step_index]

        plt.figure(figsize=figsize)
        plt.title("step_index : {}".format(step_index))
        abscisse = np.linspace(self.x_all.min(), self.x_all.max(), 1001)
        sns.histplot(self.x_all, bins=50, alpha=0.2, color="cyan", stat="probability")
        for j in range(self.nb_dist):
            plt.plot(abscisse, w[j] * stats.norm.pdf(abscisse, mu[j], sigma[j]))
        plt.show()

    def plot_params(self, figsize=(10, 7), title="Current parameters"):
        plt.figure(figsize=figsize)
        plt.title(title)
        abscisse = np.linspace(self.x_all.min(), self.x_all.max(), 1001)
        sns.histplot(self.x_all, bins=50, alpha=0.2, color="cyan", stat="probability")
        for j in range(self.w.shape[0]):
            plt.plot(abscisse, self.w[j] * stats.norm.pdf(abscisse, self.mu[j], self.sigma[j]))
        plt.show()

    def plot_progression(self, figsize=(20, 6), title="Expectation Maximization", save=True):
        abscisse = np.linspace(self.x_all.min(), self.x_all.max(), 1001)

        plt.figure(figsize=figsize)
        ax0 = plt.subplot2grid(shape=(3, 3), loc=(0, 0), rowspan=3)
        ax1_0 = plt.subplot2grid(shape=(3, 3), loc=(0, 1))
        ax1_1 = plt.subplot2grid(shape=(3, 3), loc=(1, 1))
        ax1_2 = plt.subplot2grid(shape=(3, 3), loc=(2, 1))
        ax2 = plt.subplot2grid(shape=(3, 3), loc=(0, 2), rowspan=3)

        # hist plot
        ax0.set_title("Initialization")
        mu_init = self.history["mu"][0]
        sigma_init = self.history["sigma"][0]
        w_init = self.history["w"][0]
        sns.histplot(self.x_all, bins=50, alpha=0.2, color="cyan", stat="probability", ax=ax0)

        ax2.set_title("After {} steps".format(len(self.history["mu"]) - 1))
        sns.histplot(self.x_all, bins=50, alpha=0.2, color="cyan", stat="probability", ax=ax2)

        # find y_limit
        y_lim_top = min(1, ax0.set_ylim(auto=True)[1], ax2.set_ylim(auto=True)[1])

        # line plot
        for j in range(self.nb_dist):
            ax0.plot(abscisse, w_init[j] * stats.norm.pdf(abscisse, mu_init[j], sigma_init[j]))
        for j in range(self.w.shape[0]):
            ax2.plot(abscisse, self.w[j] * stats.norm.pdf(abscisse, self.mu[j], self.sigma[j]))

        # set y_limit
        ax0.set_ylim(top=y_lim_top)
        ax2.set_ylim(top=y_lim_top)

        ax1_0.set_title("parameters")
        for i in range(self.nb_dist):
            sns.lineplot(x=np.arange(len(self.history["mu"])),
                         y=np.array(self.history["mu"])[:, i],
                         ax=ax1_0,
                         label="{}".format(i),
                         palette=sns.color_palette("hls", 8))
        ax1_0.set_ylabel("mu")
        ax1_0.set_xticks([])
        for i in range(self.nb_dist):
            sns.lineplot(x=np.arange(len(self.history["sigma"])),
                         y=np.array(self.history["sigma"])[:, i],
                         ax=ax1_1,
                         label="{}".format(i),
                         palette=sns.color_palette("hls", 8))
        ax1_1.set_ylabel("sigma")
        ax1_1.set_xticks([])
        for i in range(self.nb_dist):
            sns.lineplot(x=np.arange(len(self.history["w"])),
                         y=np.array(self.history["w"])[:, i],
                         ax=ax1_2,
                         label="{}".format(i),
                         palette=sns.color_palette("hls", 8))
        ax1_2.set_ylabel("w")
        ax1_2.set_xlabel("steps")
        x_ticks = [i for i in range(len(self.history["w"]))]
        step = len(x_ticks) // 20 + 1
        x_ticks = x_ticks[::step]
        ax1_2.set_xticks(x_ticks)

        plt.suptitle(title, fontsize=14)
        plt.tight_layout()
        if save:
            plt.savefig("../../images/1D/" + title.replace(" ", "_") + ".jpg")
        plt.show()

In [ ]:
for col in ['wind_speed', 'temperature', 'humidity', 'dew_point', 'current_precipitations']:
    print("\n\n", "=" * 50, sep="")
    print("\n\t", col)
    print("\n", "=" * 50, "\n\n", sep="")
    for k in range(2, 5):
        em = EM_1D(x=df[col].to_numpy(), nb_dist=k, split_ratio=0.01, threshold=0.001, talk=True)
        t = time.time()
        try:
            em.average_fit(runs=5, max_iters=1000, plot=True,
                           plot_title="Expectation Maximization - {} - fit {} normal distribution(s) - Average fit over 5 runs.".format(
                               col, k))
        except Exception:
            traceback.print_exc()
        finally:
            print("elapsed : {:.2f} s.".format(time.time() - t))

In [ ]:
for col in ['wind_speed', 'temperature', 'humidity', 'dew_point', 'current_precipitations']:
    print("\n\n", "=" * 50, sep="")
    print("\n\t", col)
    print("\n", "=" * 50, "\n\n", sep="")
    for k in range(1, 5):
        em = EM_1D(x=df[col].to_numpy(), nb_dist=k, split_ratio=0.01, threshold=0.001, talk=True)
        t = time.time()
        try :
            em.fit(max_iters=1000)
            em.plot_progression(title="Expectation Maximization - {} - fit {} normal distribution(s)".format(col, k))
        except Exception:
            traceback.print_exc()
        finally:
            print("elapsed : {:.2f} s.".format(time.time() - t))

## 2 dimension

source : https://towardsdatascience.com/implement-expectation-maximization-em-algorithm-in-python-from-scratch-f1278d1b9137

In [ ]:
# source : https://github.com/matplotlib/matplotlib/blob/81e8154dbba54ac1607b21b22984cabf7a6598fa/lib/matplotlib/mlab.py#L1866
def bivariate_normal(X, Y,
                     sigmax=1.0,
                     sigmay=1.0,
                     mux=0.0,
                     muy=0.0,
                     sigmaxy=0.0):
    """
    Bivariate Gaussian distribution for equal shape *X*, *Y*.
    See `bivariate normal
    <http://mathworld.wolfram.com/BivariateNormalDistribution.html>`_
    at mathworld.

    **Depreciated from matplotlib.**
    """
    Xmu = X - mux
    Ymu = Y - muy

    rho = sigmaxy / (sigmax * sigmay)
    z = Xmu ** 2 / sigmax ** 2 + Ymu ** 2 / sigmay ** 2 - 2 * rho * Xmu * Ymu / (sigmax * sigmay)
    denom = 2 * np.pi * sigmax * sigmay * np.sqrt(1 - rho ** 2)
    return np.exp(-z / (2 * (1 - rho ** 2))) / denom


class EM_2D():
    def __init__(self, x: np.ndarray, nb_dist=2, split_ratio=0.1, threshold=1e-10, minimum_sigma=1e-5, talk=False):
        self.talk = talk  # whether to display information or not
        self.threshold = threshold  # threshold ratio to trigger stop_fitting
        self.minimum_sigma = minimum_sigma  # to avoid sigma being 0
        self.stop_fitting = False  # stop fitting if maximum iterations reached or threshold passed
        self.nb_dist = nb_dist  # number of distributions to fit

        self.split_ratio = split_ratio  # part of x to keep to initialize the parameters
        self.x_all = x  # backup of whole x data
        self.idx_sample = None  # index of x_sample for initialization
        self.x_sample = None  # sample for initialization
        self.x = None  # training part of x
        self.split_x()  # fill x_sample, idx_sample and x

        self.average_by_dist_log_likelihood_record = []  # log_likelihood computed in e_step()
        self.average_log_likelihood_record = []  # log_likelihood computed in e_step()
        self.heuristics = None  # heuristics computed in e_step()

        self.mu = None  # average
        self.sigma = None  # standard deviation
        self.w = None  # weights
        self.init_params()  # initialize the above

        self.history = None  # history
        self.init_history()  # initialize history

        self.r = None  # declare r to anticipate fit()

        self.initial_record = None  # declare variable, used in average_fit()
        self.history_record = None  # declare variable, used in average_fit()
        self.clustered_distributions = None  # declare variable, used in average_fit()

        self.colors = ["blue", "red", "orange", "green", "magenta"]

    def split_x(self):
        self.idx_sample = rng.choice(np.arange(self.x_all.shape[0]),
                                     replace=False,
                                     size=int(self.x_all.shape[0] * self.split_ratio))
        self.x_sample = self.x_all[self.idx_sample]
        self.x = np.delete(self.x_all, self.idx_sample, axis=0)

    def init_params(self):
        mu = []
        for _ in range(self.nb_dist):
            mu.append(np.array(
                [rng.normal(loc=np.mean(self.x_sample[:, 0]),
                            scale=5),
                 rng.normal(loc=np.mean(self.x_sample[:, 1]),
                            scale=5)]))

        sigma = []
        for _ in range(self.nb_dist):
            s = np.zeros(shape=(2, 2))
            s[0, 0] = np.abs(rng.normal(loc=np.sqrt(np.var(self.x_sample[:, 0])),
                                        scale=2))
            s[1, 1] = np.abs(rng.normal(loc=np.sqrt(np.var(self.x_sample[:, 1])),
                                        scale=2))
            sigma.append(s)
        w = np.abs(rng.normal(loc=1,
                              scale=0.2,
                              size=self.nb_dist))
        w /= w.sum()
        if self.talk:
            print("Initial parameters :")
            print("\t - mu :    {}".format(np.round(mu, 2)))
            print("\t - sigma : {}".format(np.round(sigma, 2)))
            print("\t - w :     {}".format(np.round(w, 2)))
        self.mu, self.sigma, self.w = mu, sigma, w

    def init_history(self):
        history = {"mu": [self.mu],
                   "sigma": [self.sigma],
                   "w": [self.w]}
        self.history = history

    def update_h(self):
        self.history["mu"].append(self.mu)
        self.history["sigma"].append(self.sigma)
        self.history["w"].append(self.w)

    def e_step(self):
        np.log([stats.multivariate_normal.pdf(self.x,
                                              self.mu[i],
                                              self.sigma[i])
                for i in range(self.nb_dist)])
        log_p_y_x = np.log([self.w[i] for i in range(self.nb_dist)])[np.newaxis, ...] +\
                    np.log([stats.multivariate_normal.pdf(self.x,
                                                          self.mu[i],
                                                          self.sigma[i])
                            for i in range(self.nb_dist)]).T
        log_p_y_x_norm = logsumexp(log_p_y_x, axis=1)

        self.average_by_dist_log_likelihood_record.append(log_p_y_x.mean(axis=0))
        self.average_log_likelihood_record.append(log_p_y_x_norm.mean())  # log_likelyhood
        self.heuristics = np.exp(log_p_y_x - log_p_y_x_norm[..., np.newaxis])

    def m_step(self):
        total_count = self.x.shape[0]

        sum_heuristic = np.sum(self.heuristics, axis=0)

        w = (sum_heuristic / total_count)
        mu = []
        sigma = []
        for i in range(self.nb_dist):
            mu.append((self.heuristics[:, i][..., np.newaxis].T.dot(self.x) / sum_heuristic[i]).flatten())
            diff = self.x - mu[-1]
            sigma.append(diff.T.dot(diff * self.heuristics[:, i][..., np.newaxis]) / sum_heuristic[i])
            if np.linalg.det(sigma[-1]) == 0:
                print(
                    "WARNING : det(sigma) == 0, add minimum_sigma = {}".format(self.minimum_sigma))
                sigma[-1] += self.minimum_sigma
            if sigma[-1][0,0] == 0:
                print("WARNING : sigma[0,0] == 0, add minimum_sigma = {}".format(self.minimum_sigma))
                sigma[-1][0,0] += self.minimum_sigma
            if sigma[-1][1,1] == 0:
                print("WARNING : sigma[1,1] == 0, add minimum_sigma = {}".format(self.minimum_sigma))
                sigma[-1][1,1] += self.minimum_sigma

        self.w = w
        self.mu = mu
        self.sigma = sigma

    def fit(self, max_iters=100):
        self.stop_fitting = False
        for i in range(max_iters):
            if self.talk and i % int(max_iters // 100 + 1) == 0:
                print("{} %".format(int(i / max_iters * 100)).rjust(5), end="\r")

            self.e_step()
            self.m_step()
            self.update_h()

            if len(self.average_log_likelihood_record) > 2 and\
                    np.abs(self.average_log_likelihood_record[-1] -\
                           self.average_log_likelihood_record[-2]) < self.threshold:
                self.stop_fitting = True
                break

        if self.talk:
            print("\n")
            if self.stop_fitting:
                print("Threshold reached, less than {} % difference.".format(self.threshold * 100))
            else:
                print("100 %")
            print("Final parameters :\n\tmu :    {}\n\tsigma : {}\n\tw :     {}".format(self.history["mu"][-1],
                                                                                        self.history["sigma"][-1],
                                                                                        self.history["w"][-1]))

    def average_fit(self, runs=3, max_iters=100, plot=True, plot_title=""):
        """
        to avoid initialization issues, average over [runs] fit() and different initializations.
        :param plot_title: title of the plot of plot
        :param runs: number of fit() to execute
        :param max_iters: maximum number of iterations for each fit()
        :param plot: plot the final result, average of all fit()

        updates to history to every parameters fo every fit() (for plotting purpose) and parameters to average of every run.
        """
        raise NotImplementedError("not implemented yet.")

    ###########################
    ### POT FUNCTIONS BELOW ###
    ###########################

    def get_contour_parameters(self):
        # source : https://stackoverflow.com/questions/37890550/python-plotting-percentile-contour-lines-of-a-probability-distribution
        min_x = self.x_all[:, 0].min()
        max_x = self.x_all[:, 0].max()
        min_y = self.x_all[:, 1].min()
        max_y = self.x_all[:, 1].max()
        extent = [min_x, max_x, min_y, max_y]

        X, Y = np.mgrid[min_x:max_x:200j, min_y:max_y:200j]

        for i in range(self.nb_dist):
            z = self.w[i] * bivariate_normal(X, Y,
                                             mux=self.mu[i][0],
                                             muy=self.mu[i][1],
                                             sigmax=self.sigma[i][0, 0],
                                             sigmay=self.sigma[i][1, 1],
                                             sigmaxy=self.sigma[i][1, 0])
            z = z / z.sum()
            n = 1000
            t = np.linspace(0, z.max(), n)
            integral = ((z >= t[:, None, None]) * z).sum(axis=(1, 2))

            f = interpolate.interp1d(integral, t)
            t_contours = f(np.array([0.9, 0.7, 0.5, 0.3, 0.1]) / 5)
            yield z, t_contours, extent

    def plot_2D(self, cols=("", ""), scatter=False, contour=True, progression=True, height=10,
                title="Expectation Maximization 2D",
                save=False, show=True):
        h = sns.jointplot(x=self.x_all[:, 0],
                          y=self.x_all[:, 1],
                          kind="hex",
                          marginal_kws={"bins": 100},
                          height=height)
        h.set_axis_labels(*cols)
        h.figure.tight_layout()

        if scatter:
            for i in range(self.nb_dist):
                norm = rng.multivariate_normal(self.mu[i], self.sigma[i], size=(2, 10000))[0]
                sns.scatterplot(x=norm[:, 0], y=norm[:, 1], alpha=0.1, color=self.colors[i])

        if contour:
            generator_contour = self.get_contour_parameters()
            for i in range(self.nb_dist):
                z, t_contours, extent = next(generator_contour)
                # plt.imshow(z.T, origin='lower', extent=[min_x,max_x,min_y,max_y], cmap="gray")
                plt.contour(z.T, t_contours, extent=extent, colors=self.colors[i])

        if progression:
            line_mu = {i: [] for i in range(self.nb_dist)}
            for i in range(self.nb_dist):
                for j in range(len(self.history["mu"])):
                    line_mu[i].append(self.history["mu"][j][i])
                line_mu[i] = np.array(line_mu[i])
                plt.plot(line_mu[i][:, 0],
                         line_mu[i][:, 1],
                         marker="o",
                         color="magenta",
                         markersize=2,
                         linewidth=1)

        if scatter or contour or progression:
            for i in range(self.nb_dist):
                plt.text(x=self.mu[i][0] - 1,
                         y=self.mu[i][1],
                         s="w:" + str(round(self.w[i], 2)),
                         color=self.colors[i],
                         bbox=dict(facecolor='white', alpha=0.5),
                         )

        plt.title(title, y=.97)
        if save:
            plt.savefig("../../images/2D/" + title.replace(" ", "_") + ".jpg")
        if show:
            plt.show()
        else :
            plt.close(h)


    def plot_likelihood(self, figsize=(10, 5), title="likelihood", save=False, show=True):
        plt.figure(figsize=figsize)
        log_likelihood = np.array(self.average_by_dist_log_likelihood_record)
        for i in range(self.nb_dist):
            sns.lineplot(x=np.arange(0, log_likelihood.shape[0]),
                         y=log_likelihood[:, i],
                         color=self.colors[i])
        sns.lineplot(x=np.arange(0, len(self.average_log_likelihood_record)),
                     y=self.average_log_likelihood_record,
                     color="black")
        plt.legend(["distrib. " + str(i) for i in range(self.nb_dist)] + ["Global"])
        plt.xlabel("steps")
        plt.ylabel("log_likelihood")
        plt.title(title)
        if save:
            plt.savefig("../../images/2D/" + title.replace(" ", "_") + ".jpg")
        if show:
            plt.show()
        else :
            plt.cls()

    def plot_original(self, cols=("", ""), title="Original", save=False, show=True):
        h = sns.jointplot(x=self.x_all[:, 0],
                          y=self.x_all[:, 1],
                          kind="hex",
                          marginal_kws={"bins": 100},
                          height=10)
        h.set_axis_labels(*cols)
        h.figure.tight_layout()
        plt.title(title, y=.97)
        if save:
            plt.savefig("../../images/2D/" + title.replace(" ", "_") + ".jpg")
        if show:
            plt.show()
        else :
            plt.close(h)

In [ ]:
columns = ['wind_speed', 'temperature', 'humidity', 'dew_point', 'current_precipitations']
show = False
for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        plot_original = True
        col1 = columns[i]
        col2 = columns[j]
        for k in range(2, 5):
            for run in range(1, 4):
                t=time.time()
                title = f"{run}/Expectation Maximization 2D - {col1} and {col2}"
                em2 = EM_2D(x=df[[col1, col2]].to_numpy(), nb_dist=k)
                if plot_original:
                    #plot original distribution
                    em2.plot_original(title=title + " - original", cols=[col1, col2], save=True, show=False)
                    plot_original = False
                try :
                    #fit
                    em2.fit(max_iters=300)
                    #plot fit
                    em2.plot_2D(title=title + f" - fit {k} normal distributions", cols=[col1, col2], scatter=True,
                                contour=True, progression=True, save=True, show=False)
                    #plot likelihood
                    em2.plot_likelihood(title=title + f" - fit {k} normal distributions - likelihood", save=True,
                                        show=False)
                except Exception:
                    traceback.print_exc()
                finally:
                    print("elapsed : {:.2f} s.".format(time.time() - t))